## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 4% des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

In [49]:
def analyse_integrite(data, fields, pr=True):
    '''analyse les relations du DataFrame définies dans le dict fields et retourne un dict avec pour chaque contrôle la
    liste des index ko. Les résultats des contrôles sont également ajoutés sous forme de champs booléens à data'''
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        if pr:
            print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

def add_nbre_pdc(data, name='ok', pr=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    #data[name] = data[name] & data['nbre_pdc_ok']
    if pr:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

------
# 2 - Initialisation
## initialisation des données
- lecture du fichier issu de l'api

In [3]:
from datetime import datetime
import json
from observation import Ilist, Analysis
import pandas as pd
#date = '2023-03-03'
#date = '2023-04-17'
date = '2023-05-02'
log = {'date_irve': date, 
       'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
      'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
#      'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\'}
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  70625


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
fields = [
 
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory =  ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
           'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
           'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
           'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
           'date_maj', 'last_modified']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- un peu moins de 1 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  654
nombre de pdc en itinerance   :  69971


In [6]:
# arborescence des champs et nombre de valeurs différentes
il = Ilist.obj(itinerance_init)
print(il.tree())

-1: root-derived (69971)
   0 : index (69971)
      1 : contact_operateur (126)
      2 : nom_enseigne (1144)
      3 : coordonneesXY (17399)
      4 : adresse_station (16684)
      5 : id_station_itinerance (19095)
      6 : nom_station (15629)
      7 : implantation_station (5)
      8 : nbre_pdc (52)
      9 : condition_acces (2)
      10: horaires (512)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (47006)
      13: date_maj (598)
      14: last_modified (515)


-----------------------
## 4 - Bilan initial intégrité
- plus de 50 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          32391
contact_operateur - id_station_itinerance          12919
nom_enseigne - id_station_itinerance                9462
coordonneesXY - id_station_itinerance              14190
id_station_itinerance - id_pdc_itinerance            385
nom_station - id_station_itinerance                 2111
implantation_station - id_station_itinerance        1294
nbre_pdc - id_station_itinerance                    4902
condition_acces - id_station_itinerance               31
horaires - id_station_itinerance                   12160
station_deux_roues - id_station_itinerance         13742
adresse_station - coordonneesXY                     1483

nombre d'enregistrements sans erreurs :  34384
nombre d'enregistrements avec au moins une erreur :  35587
taux d'erreur :  51  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- plus de la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de plus de 80% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  22965  soit  65  %
nombre de pdc sans doublon   :  12622


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            270
nom_enseigne - id_station_itinerance                 452
coordonneesXY - id_station_itinerance               1081
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  558
implantation_station - id_station_itinerance         237
nbre_pdc - id_station_itinerance                    1008
condition_acces - id_station_itinerance               13
horaires - id_station_itinerance                     290
station_deux_roues - id_station_itinerance           719
adresse_station - coordonneesXY                      641

nombre d'enregistrements sans erreurs :  9544
nombre d'enregistrements avec au moins une erreur :  3078


-----------------------
## 6 - Séparation doublons station - date de maj
- plus de 15% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de plus de 20% le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 32 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['etape3_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  512  soit  17  %
nombre de pdc sans doublon   :  2566


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                 267
coordonneesXY - id_station_itinerance                477
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  328
implantation_station - id_station_itinerance          64
nbre_pdc - id_station_itinerance                     871
condition_acces - id_station_itinerance                8
horaires - id_station_itinerance                      75
station_deux_roues - id_station_itinerance           451
adresse_station - coordonneesXY                      440

nombre d'enregistrements sans erreurs :  218
nombre d'enregistrements avec au moins une erreur :  2348  soit  76  %


In [12]:
# structure des données résiduelles et nombre de valeurs
il = Ilist.obj(itinerance_5)
print(il.tree())

-1: root-derived (2348)
   0 : index (2348)
      1 : contact_operateur (32)
      2 : nom_enseigne (325)
      3 : coordonneesXY (582)
      4 : adresse_station (632)
      5 : id_station_itinerance (567)
         13: date_maj (85)
      6 : nom_station (600)
      7 : implantation_station (5)
      8 : nbre_pdc (21)
      9 : condition_acces (2)
      10: horaires (91)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (2348)
      14: last_modified (59)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  29


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [14]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  2348
total des doublons à supprimer :  23477
nombre de pdc avec controles ok :  44146


In [15]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [16]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, pr=False)
log['bilan_erreurs'] = sum([len(controle) for controle in res.values()])
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [17]:
# structure des données bonnes
il = Ilist.obj(itinerance_ok.loc[:, mandatory])
print(il.tree())

-1: root-derived (44146)
   11: id_pdc_itinerance (44146)
      4 : id_station_itinerance (18465)
         1 : nom_enseigne (825)
         2 : coordonneesXY (14570)
            0 : contact_operateur (109)
            3 : adresse_station (13976)
               8 : condition_acces (2)
         5 : nom_station (14619)
         6 : implantation_station (5)
         7 : nbre_pdc (52)
         9 : horaires (427)
         10: station_deux_roues (5)
         19: paiement_autre (7)
      12: puissance_nominale (81)
      13: prise_type_ef (8)
      14: prise_type_2 (8)
      15: prise_type_combo_ccs (7)
      16: prise_type_chademo (7)
      17: prise_type_autre (7)
      18: paiement_acte (8)
      20: reservation (6)
      21: accessibilite_pmr (4)
      22: restriction_gabarit (95)
      23: date_maj (455)
      24: last_modified (371)


### Indicateurs

In [18]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [43]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  80  % des écarts

hello@powerdot.fr 618
supervision-ev.france@totalenergies.com 506
contact@e55c.com 284
pascal.lhermitte@siege27.fr 258
web@freshmile.com 202


In [54]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, pr=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1208
51  %


In [57]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 29 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [20]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1075,38021,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299331,2022-11-03,2023-05-02T09:29:58.135000+00:00
1078,38025,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299351,2022-11-03,2023-05-02T09:29:58.135000+00:00
1142,38107,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.533698, 43.251851]",PLACE AVENUE GENERAL DE GAULLE -COGOLIN,FR55CP83310,PLACE AVENUE GENERAL DE GAULLE -COGOLIN,Voirie,2,Accès réservé,24/7,FALSE,FR55CE833104325191965336561,2022-11-03,2023-05-02T09:29:58.135000+00:00
1144,38110,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.577631, 43.279636]",PARKING AIRE DE ST PONS - GRIMAUD,FR55CP83310,PARKING AIRE DE ST PONS - GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CE83310GIRVATMX029BAD1,2022-11-03,2023-05-02T09:29:58.135000+00:00
1145,38111,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.577631, 43.279636]",PARKING AIRE DE ST PONS - GRIMAUD,FR55CP83310,PARKING AIRE DE ST PONS - GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CE83310GIRVATMX029BAD2,2022-11-03,2023-05-02T09:29:58.135000+00:00
1146,38112,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.577631, 43.279636]",PARKING AIRE DE ST PONS - GRIMAUD,FR55CP83310,PARKING AIRE DE ST PONS - GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CE83310GIRVATMX029BAD3,2022-11-03,2023-05-02T09:29:58.135000+00:00
1167,38146,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.522957, 43.274431]",PARKING MAIRIE - GRIMAUD,FR55CP83310,PARKING MAIRIE - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00051,2022-11-03,2023-05-02T09:29:58.135000+00:00
1168,38147,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.522957, 43.274431]",PARKING MAIRIE - GRIMAUD,FR55CP83310,PARKING MAIRIE - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00052,2022-11-03,2023-05-02T09:29:58.135000+00:00
1169,38148,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.57743, 43.274761]",PARKING PORT - GRIMAUD,FR55CP83310,PARKING PORT - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00071,2022-11-03,2023-05-02T09:29:58.135000+00:00
1170,38149,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.57743, 43.274761]",PARKING PORT - GRIMAUD,FR55CP83310,PARKING PORT - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00072,2022-11-03,2023-05-02T09:29:58.135000+00:00


### Cohérence implantation_station - id_station
- 64 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [21]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
714,19519,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE205868TE,2022-07-25,2023-05-02T09:30:52.214000+00:00
717,19646,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE200921,2022-07-25,2023-05-02T09:30:52.214000+00:00
718,19647,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE201426,2022-07-25,2023-05-02T09:30:52.214000+00:00
719,19671,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE201290,2022-07-25,2023-05-02T09:30:52.214000+00:00
721,19724,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE200977,2022-07-25,2023-05-02T09:30:52.214000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,38365,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2023-05-02T09:29:58.135000+00:00
1314,38366,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2023-05-02T09:29:58.135000+00:00
1315,38367,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2023-05-02T09:29:58.135000+00:00
1316,38368,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2023-05-02T09:29:58.135000+00:00


### Cohérence nom_station - id_station
- 328 pdc sont associés à une station avec un nom non cohérent

In [22]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
333,13709,service-recharge@soregies.fr,ALTERBASE86,"[0.37178, 46.56869]","78, avenue Jacques Coeur - 86000 - Poitiers",FRS86PSOREB140,Sorégies 3,Parking privé à usage public,2,Accès libre,24/7,false,FRS86ESOREB1402,2020-04-30,2023-05-02T09:33:33.763000+00:00
334,13723,service-recharge@soregies.fr,ALTERBASE86,"[0.37178, 46.56869]","78, avenue Jacques Coeur - 86000 - Poitiers",FRS86PSOREB140,Sorégies 2,Parking privé à usage public,2,Accès libre,24/7,false,FRS86ESOREB1401,2020-04-30,2023-05-02T09:33:33.763000+00:00
531,18386,pascal.lhermitte@siege27.fr,SIEGE50,"[1.221452, 49.197647]",Ecoparc satellite ZA1 Heudebouville,FRS27PHEUDEBOUVILLEECOPARC,900012,Voirie,2,Accès libre,24/7,false,FRS27EHEUDEBOUVILLECOPARC1G,2021-06-15,2023-05-02T09:30:56.425000+00:00
532,18387,pascal.lhermitte@siege27.fr,SIEGE49,"[1.221452, 49.197647]",Ecoparc satellite ZA1 Heudebouville,FRS27PHEUDEBOUVILLEECOPARC,900012,Voirie,2,Accès libre,24/7,false,FRS27EHEUDEBOUVILLECOPARC1D,2021-06-15,2023-05-02T09:30:56.425000+00:00
561,18416,pascal.lhermitte@siege27.fr,SIEGE88,"[1.224898, 49.195263]",Ecoparc satellite ZA2 Heudebouville,FRS27PHEUDEBOUVILLEECOPARC,900031,Voirie,2,Accès libre,24/7,false,FRS27EHEUDEBOUVILLECOPARC2G,2021-06-15,2023-05-02T09:30:56.425000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,64645,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM016,2023-04-13,2023-05-02T09:28:07.917000+00:00
2079,64670,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM017,2023-04-13,2023-05-02T09:28:07.917000+00:00
2080,64671,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM019,2023-04-13,2023-05-02T09:28:07.917000+00:00
2081,64673,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM020,2023-04-13,2023-05-02T09:28:07.917000+00:00


### Cohérence adresse - coordonnées
- 440 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [23]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4,342,web@freshmile.com,Freshmile,"[4.08079663, 48.29366661]",Parking cinéma culturel 10150 Pont Sainte Marie,FRS10PC16QZJT72Y,"Pont-Sainte-Marie, Parking cinéma culturel",Voirie,2,Accès libre,24/7,false,FRS10EGCLC1,2023-04-28,2023-05-02T10:59:48.917000+00:00
5,346,web@freshmile.com,Freshmile,"[4.08079663, 48.29366661]",Parking cinéma culturel 10150 Pont Sainte Marie,FRS10PC16QZJT72Y,"Pont-Sainte-Marie, Parking cinéma culturel",Voirie,2,Accès libre,24/7,false,FRS10EGCLC2,2023-04-28,2023-05-02T10:59:48.917000+00:00
6,349,web@freshmile.com,Freshmile,"[4.08079663, 48.29366661]",Parking cinéma culturel 10150 Pont-Sainte-Marie,FRS10PJJYAMZXUCU,"Pont-Sainte-Marie, Parking cinéma culturel",Voirie,2,Accès libre,24/7,false,FRS10EUXPG1,2023-05-02,2023-05-02T10:59:48.917000+00:00
13,1870,web@freshmile.com,Freshmile,"[55.29748, -20.9429]",89 rue du Général de Gaulle 97420 La Port,FRFR1PEQRCK1OHOC,"Engen, Oasis - Le Port IES",Voirie,2,Accès libre,24/7,false,FRFR1EMBYM2,2023-05-02,2023-05-02T10:59:48.917000+00:00
14,1877,web@freshmile.com,Freshmile,"[55.29748, -20.9429]",89 rue du Général de Gaulle 97420 La Port,FRFR1PEQRCK1OHOC,"Engen, Oasis - Le Port IES",Voirie,2,Accès libre,24/7,false,FRFR1EMBYM1,2023-05-02,2023-05-02T10:59:48.917000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,65203,supervision-ev.france@totalenergies.com,MERCURE ALBI,"[2.146379, 43.932802]",44 bis rue porta ALBI 81000,FRTCBP01185,Mercure Albi Bastides,Parking privé à usage public,4,Accès libre,24/7,false,FRTCBE006334,2022-11-29,2023-05-02T09:28:07.917000+00:00
2562,69199,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2023-04-24,2023-05-02T09:27:52.363000+00:00
2563,69228,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2023-04-24,2023-05-02T09:27:52.363000+00:00
2564,69261,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2023-04-24,2023-05-02T09:27:52.363000+00:00


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [25]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1878,65152,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-05-02T09:28:07.917000+00:00
1879,65153,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-05-02T09:28:07.917000+00:00
1880,65154,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-05-02T09:28:07.917000+00:00
1881,65155,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-05-02T09:28:07.917000+00:00


### Cohérence station - coordonnées
- 477 pdc sont associés à des stations avec plusieurs coordonnées

In [26]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1100,38053,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.805081, 43.936094]",MONCLAR - AVIGNON,FR55CP84000,MONCLAR - AVIGNON,Voirie,2,Accès réservé,24/7,FALSE,FR55CE840004393609448050821,2022-11-03,2023-05-02T09:29:58.135000+00:00
1101,38054,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.974066, 45.698131]",BP - SAINT-PRIEST,FR55CPBP698,BP - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0234FA2,2022-11-03,2023-05-02T09:29:58.135000+00:00
1102,38055,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.899412, 43.940189]",AVENUE JEAN MONNET - MORIERE,FR55CP84310,AVENUE JEAN MONNET - MORIERE,Voirie,2,Accès réservé,24/7,FALSE,FR55CE843104394018948994121,2022-11-03,2023-05-02T09:29:58.135000+00:00
1103,38056,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239151,2022-11-03,2023-05-02T09:29:58.135000+00:00
1104,38057,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239152,2022-11-03,2023-05-02T09:29:58.135000+00:00
1105,38058,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239153,2022-11-03,2023-05-02T09:29:58.135000+00:00
1106,38065,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.974066, 45.698131]",BP - SAINT-PRIEST,FR55CPBP698,BP - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0234FA1,2022-11-03,2023-05-02T09:29:58.135000+00:00
1107,38066,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.974066, 45.698131]",BP - SAINT-PRIEST,FR55CPBP698,BP - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0234FA3,2022-11-03,2023-05-02T09:29:58.135000+00:00
1108,38067,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.843686, 43.946211]",PARKING PONT DES DEUX EAUX - AVIGNON,FR55CP84000,PARKING PONT DES DEUX EAUX - AVIGNON,Voirie,4,Accès réservé,24/7,FALSE,FR55CE840004394621148436841,2022-11-03,2023-05-02T09:29:58.135000+00:00
1109,38068,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-05-02T09:29:58.135000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [27]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1810,51073,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ETCIX5105,2022-07-21,2023-05-02T09:29:11.021000+00:00
1811,51074,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EREUO5946,2022-07-21,2023-05-02T09:29:11.021000+00:00
1812,51075,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-05-02T09:29:11.021000+00:00
1813,51076,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-05-02T09:29:11.021000+00:00
1814,51077,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-05-02T09:29:11.021000+00:00
1815,51078,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-05-02T09:29:11.021000+00:00
1816,51079,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-05-02T09:29:11.021000+00:00
1817,51080,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-05-02T09:29:11.021000+00:00
1818,51081,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EYLF65474,2022-07-21,2023-05-02T09:29:11.021000+00:00
1819,51083,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-05-02T09:29:11.021000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [44]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
906,38068,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-05-02T09:29:58.135000+00:00
907,38069,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-05-02T09:29:58.135000+00:00
908,38070,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-05-02T09:29:58.135000+00:00
909,38071,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-05-02T09:29:58.135000+00:00
910,38072,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-05-02T09:29:58.135000+00:00
911,38073,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-05-02T09:29:58.135000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [45]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2344,69199,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2023-04-24,2023-05-02T09:27:52.363000+00:00
2345,69228,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2023-04-24,2023-05-02T09:27:52.363000+00:00
2346,69261,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2023-04-24,2023-05-02T09:27:52.363000+00:00
2347,69263,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649711,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649711,2023-04-24,2023-05-02T09:27:52.363000+00:00


### Incohérence 'nbre_pdc'
- 25% des pdc ont un champ 'nbre_pdc' mal documenté

- exemple : station avec un pdc 'de regroupement' qui évite de documenter plusieurs lignes

In [30]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLUMEACACIAS11', 
               ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
19558,FRLUMEACACIAS11,FRLUMEACACIAS11,"[3.1485441,50.7582304]",4


- exemple : station avec plusieurs pdc mais l'id_station est identique à l'id_pdc

In [31]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[2.451322, 45.66523]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
16143,FRSDGEMBDG512,FRSDGPMBDG512,"[2.451322, 45.66523]",2
16144,FRSDGEMBDG511,FRSDGPMBDG511,"[2.451322, 45.66523]",2


- exemple : station avec plusieurs pdc mais le champs a toujours une valeur de 1

In [32]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[-1.7548354193520386, 48.125067030488154]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
13356,FRIENE35353A21,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
13357,FRIENE35353A31,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
13358,FRIENE35353A11,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
13359,FRIENE35353A41,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
13360,FRIENE35353A51,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1


### cohérence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [33]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PBARREOUCHEMOMORT', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18132,18509,pascal.lhermitte@siege27.fr,SIEGE266,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTG,2021-06-15,2023-05-02T09:30:56.425000+00:00
18133,18510,pascal.lhermitte@siege27.fr,SIEGE265,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTD,2021-06-15,2023-05-02T09:30:56.425000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [34]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
17919,18289,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:01,False,FRS23P230200101,2023-03-09,2023-05-02T09:31:04.537000+00:00
17920,18290,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:02,False,FRS23P230200102,2023-03-09,2023-05-02T09:31:04.537000+00:00


### cohérence coordonnées - adresse
- exemple de plusieurs stations avec des coordonnées identiques mais des adresses différentes

In [35]:
itinerance_5.loc[itinerance_5.coordonneesXY == '[2.460441, 50.78763]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
331,18197,support@passpasselectrique.fr,Pass Pass Electrique,"[2.460441, 50.78763]",1bis Rue De La Gare 59670 Bavinchove,FRH14E59054001,Bavinchove - Gare,Parking public,2,Accès libre,24/7,false,FRH14E59054001,2023-01-27,2023-05-02T09:31:13.232000+00:00
713,23477,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540013,2023-03-17,2023-05-02T09:30:20.698000+00:00
714,23481,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540012,2023-03-17,2023-05-02T09:30:20.698000+00:00
742,24644,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540014,2023-03-17,2023-05-02T09:30:20.698000+00:00
745,24848,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540011,2023-03-17,2023-05-02T09:30:20.698000+00:00
